In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [118]:
# Load data
path = '/home/han/repos/BEACO2N-CO2-Correction/data/'
X_train  = torch.Tensor(np.load(path + 'X_train.npy', allow_pickle=True))
X_test   = torch.Tensor(np.load(path + 'X_test.npy', allow_pickle=True))
X_val    = torch.Tensor(np.load(path + 'X_val.npy', allow_pickle=True))
y_train  = torch.Tensor(np.load(path + 'y_train.npy', allow_pickle=True))
y_test   = torch.Tensor(np.load(path + 'y_test.npy', allow_pickle=True))
y_val    = torch.Tensor(np.load(path + 'y_val.npy', allow_pickle=True))

In [119]:
# Parameters
n_iters      = 50
nsize_in     = X_train.shape[1] 
nsize_out    = 1
nsize_hidden = X_train.shape[1]
n_layers     = 4
activations = [nn.ReLU(),
               nn.ReLU(),
               nn.ReLU(), 
               None]
layer_combiners = [nn.Linear(nsize_in, nsize_hidden),
                   nn.Linear(nsize_in, nsize_hidden),
                   nn.Linear(nsize_in, nsize_hidden),
                   nn.Linear(nsize_in, nsize_out)]
learning_rate = 0.05

In [120]:
def create_model(n_layers, activation, combiners):
    
    if len(activation) != n_layers:
        act = activation[0]*(n_layers - 1)
        act.append(activation[-1])
    else:
        act = activations
    if len(combiners) != n_layers:
        combine = combiners[0]*(n_layers - 1)
        combine.append(combiners[-1])
    else:
        combine = combiners
        
    for layer in range(n_layers):
        if layer == 0:
            model = nn.Sequential(combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
        else:
            model.add_module(name=str(2*layer),module=combine[layer])
            if act[layer]:
                model.add_module(name=str(2*layer+1), module=act[layer])
                
    return model

In [121]:
try:
    del model
except:
    pass
model = create_model(n_layers, activations, layer_combiners)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [122]:
for epoch in range(n_iters):
   # Forward propagation
    y_pred = model(X_train)
    
    #Compute and print loss
    loss  = criterion(y_pred, y_train)
    print('epoch: ',epoch, ' loss: ', loss.item())
    
    #Zero the gradients (so that gradients don't accumulate from epoch to epoch)
    optimizer.zero_grad()
    
    #perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()

epoch:  0  loss:  188947.09375
epoch:  1  loss:  inf
epoch:  2  loss:  inf
epoch:  3  loss:  nan
epoch:  4  loss:  nan
epoch:  5  loss:  nan
epoch:  6  loss:  nan
epoch:  7  loss:  nan
epoch:  8  loss:  nan
epoch:  9  loss:  nan
epoch:  10  loss:  nan
epoch:  11  loss:  nan
epoch:  12  loss:  nan
epoch:  13  loss:  nan
epoch:  14  loss:  nan
epoch:  15  loss:  nan
epoch:  16  loss:  nan
epoch:  17  loss:  nan
epoch:  18  loss:  nan
epoch:  19  loss:  nan
epoch:  20  loss:  nan
epoch:  21  loss:  nan
epoch:  22  loss:  nan
epoch:  23  loss:  nan
epoch:  24  loss:  nan
epoch:  25  loss:  nan
epoch:  26  loss:  nan
epoch:  27  loss:  nan
epoch:  28  loss:  nan
epoch:  29  loss:  nan
epoch:  30  loss:  nan
epoch:  31  loss:  nan
epoch:  32  loss:  nan
epoch:  33  loss:  nan
epoch:  34  loss:  nan
epoch:  35  loss:  nan
epoch:  36  loss:  nan
epoch:  37  loss:  nan
epoch:  38  loss:  nan
epoch:  39  loss:  nan
epoch:  40  loss:  nan
epoch:  41  loss:  nan
epoch:  42  loss:  nan
epoch:  43  

In [86]:
# Lets Evaluate
model.eval()
print(model.training ) #Just to be sure

ypred = model(X_test)
loss  = criterion(y_pred, y_test)
print("Loss: ", loss)

False
Loss:  tensor(443.4062, grad_fn=<MseLossBackward>)


/home/han/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([6327])) that is different to the input size (torch.Size([18981, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [89]:
ypred = model(X_test)
err = torch.norm(ypred-y_test, p=2)
print(y_test[1:10])
print(ypred[1:10])
print(err)

tensor([410.6907, 441.9533, 413.5955, 415.1597, 414.4968, 413.9651, 419.0463,
        410.7672, 407.9546])
tensor([[423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161],
        [423.4161]], grad_fn=<SliceBackward>)
tensor(131980.1406, grad_fn=<NormBackward0>)


In [90]:
y_train[1:10]

tensor([433.2132, 410.1967, 465.3893, 432.4657, 417.1220, 416.7152, 413.6294,
        413.1808, 420.4663])

In [117]:
torch.mean(X_train,0)

tensor([-1.7644e-01, -5.6377e-02, -7.0168e-02,  6.6849e-03,  3.0164e-04,
        -5.8091e+00, -3.0789e+02])